[Reference](https://towardsdatascience.com/raw-materials-optimization-for-food-manufacturing-with-python-fbf2be4a74)

# I. Problem Statement

<b> Scenario </b> <br>
7 ingredients are available
- Meat: Chicken, Beef, Mutton
- Non-Meat: Rice, Corn, Wheat bran, Peanuts

<b> Objective </b> <br>
Minimize the total cost per bar (Weight: 120g).

<b> Constraints </b> <br>
Minimal values for protein and fibre <br>
Maximum values for fat, salt and sugar




# II. Build your model


In [2]:
pip install pulp

     |████████████████████████████████| 41.2 MB 72 kB/s 


## 1. Import Parameters and Declare your variables, parameters and model


In [8]:
dict_costs

{'Beef': 0.15,
 'Chicken': 0.095,
 'Corn': 0.012,
 'Mutton': 0.1,
 'Peanuts': 0.013000000000000001,
 'Rice': 0.002,
 'Wheat bran': 0.005}

In [3]:
import pandas as pd
from pulp import *

# Import Nutrition Facts
nutrition = pd.read_csv('https://gist.githubusercontent.com/samirsaci/b787a80c5d4c753c14903d1c01ad0501/raw/7c311689985303c16947eb023936b91adbf5b0f7/nutritions_facts.csv', index_col = 0)
# Import Costs
costs = pd.read_csv('https://gist.githubusercontent.com/samirsaci/9d205ca300ab33b1e449ac593d84f851/raw/f923fbb6c9007ea26f6707f1f5f8843737305089/costs.csv')
dict_costs = dict(zip(costs['Ingredients'], costs['Costs']))

# Variables
variables = ['Chicken', 'Beef', 'Mutton', 'Rice', 'Wheat bran', 'Corn', 'Peanuts']

# Initialize Class
model = LpProblem("Optimize your Protein Bar", LpMinimize)

# Create Decision Variables
x = LpVariable.dicts("Qty", [j for j in variables],
                     lowBound=0, upBound=None, cat='continuous')

/usr/local/lib/python3.7/dist-packages/pulp/pulp.py:1313: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


## 2. Define the objective and add constraints


In [4]:
# Define Objective Function
model += (lpSum([dict_costs[i] * x[i] for i in variables]))


# Add Constraints
model += (lpSum([x[i] for i in variables])) == 120
model += (lpSum([x[i] * nutrition.loc[i, 'Protein'] for i in variables])) >= 22
model += (lpSum([x[i] * nutrition.loc[i, 'Fat'] for i in variables])) <= 22
model += (lpSum([x[i] * nutrition.loc[i, 'Fibre'] for i in variables])) >= 6
model += (lpSum([x[i] * nutrition.loc[i, 'Salt'] for i in variables])) <= 3
model += (lpSum([x[i] * nutrition.loc[i, 'Sugar'] for i in variables])) <= 20

# 3. Solve your model and analyze the results

In [5]:
# Solve Model
model.solve()
print("Cost per Bar = {:,} $".format(round(value(model.objective), 2)))
print('\n' + "Status: {}".format(LpStatus[model.status]))
for v in model.variables():
    print(v.name, "=", round(v.varValue,2), 'g')

Cost per Bar = 10.32 $

Status: Optimal
Qty_Beef = 65.32 g
Qty_Chicken = 0.0 g
Qty_Corn = 0.0 g
Qty_Mutton = 0.0 g
Qty_Peanuts = 30.96 g
Qty_Rice = 0.0 g
Qty_Wheat_bran = 23.72 g
